In [19]:
from edges_cal.loss import get_cable_loss_model
from edges_cal.ee import KNOWN_CABLES
import numpy as np
from astropy import units as u
import attrs
import ctypes

In [30]:
from ctypes import cdll

antloss_lib = cdll.LoadLibrary("antloss.so")
lossmodel = antloss_lib.lossmodel
lossmodel.restype = ctypes.c_double
lossmodel.argtypes = [ctypes.c_double, ctypes.c_double, ctypes.c_double, ctypes.c_int]

In [36]:
alan = lossmodel(50.0, 0.0, 0.0, 6)

SKIN DEPTH: 0.420815 0.133073
PROP CONSTANT: 0.018761 1.519219
G: 6.127692292683e-06
C: 9.752525181235e-11
L: 2.397505225929e-07
R: 1.845450278693e+00
s11: 5.305484346595e-04 -4.071350218314e-04
s12: 9.984042141051e-01 -4.553381508384e-02
ta11: 9.984040787450e-01 $-4.553338855696e-02
ta12: 5.488525568679e-04 -3.827544651754e-04
ta21: -5.488525568679e-04 3.827544651754e-04
ta22 9.995193748142e-01 4.558467376493e-02
SKIN DEPTH: 0.121059 0.065192
PROP CONSTANT: 0.001274 1.085243
G: 0.000000000000e+00
C: 6.799273664478e-11
L: 1.755057936671e-07
R: 1.294481303388e-01
s11: 1.428786235488e-02 4.343184802218e-03
s12: 3.600653167206e-01 -9.312907176085e-01
tb11: 3.601143356098e-01 -9.315086192310e-01
tb12: 1.103161260231e-03 1.491548448157e-02
tb21: -1.103161260231e-03 -1.491548448157e-02
tb22 3.611670810964e-01 9.341403754028e-01
t11: 3.171185631426e-01 -9.464269949564e-01
t12: 2.336327127845e-03 1.521591689310e-02
t21: -2.638241495113e-04 -1.430950671289e-02
t22 3.184046963964e-01 9.501473234

In [32]:
balun = KNOWN_CABLES['lowband-balun-tube']
balun = attrs.evolve(balun, eps0=8.854e-12*u.F/u.m)

In [33]:
connector = KNOWN_CABLES['SC3792 Connector']
connector = attrs.evolve(connector, eps0=8.854e-12*u.F/u.m)

In [34]:
mdl = get_cable_loss_model([connector, balun])

In [37]:
mdl(50*u.MHz, 0.0), alan

(array([0.99605126]), 0.9960512573205863)

In [40]:
mdl(40*u.MHz, 1e-3 + 1e-2j), lossmodel(40.0, 1e-3, 1e-2, 6)

SKIN DEPTH: 0.420815 0.133073
PROP CONSTANT: 0.016742 1.217153
G: 4.902153834147e-06
C: 9.752525181235e-11
L: 2.404438838318e-07
R: 1.650620908901e+00
s11: 4.815630810937e-04 -2.716091236734e-04
s12: 9.988351812631e-01 -3.648593049296e-02
ta11: 9.988350136042e-01 $-3.648567471837e-02
ta12: 4.914020150893e-04 -2.539756995603e-04
ta21: -4.914020150893e-04 2.539756995603e-04
ta22 9.998320713757e-01 3.652234537305e-02
SKIN DEPTH: 0.121059 0.065192
PROP CONSTANT: 0.001139 0.868315
G: 0.000000000000e+00
C: 6.799273664478e-11
L: 1.755544291232e-07
R: 1.157819275991e-01
s11: 1.145599068407e-02 6.667603273227e-03
s12: 5.713881587152e-01 -8.190252246560e-01
tb11: 5.714638984052e-01 -8.191840226359e-01
tb12: 1.087833723065e-03 1.322842697623e-02
tb21: -1.087833723065e-03 -1.322842697623e-02
tb22 5.729426974699e-01 8.212534935366e-01
t11: 5.409057746908e-01 -8.390861544967e-01
t12: 2.059338121872e-03 1.343137779063e-02
t21: -6.772835418857e-04 -1.271824916081e-02
t22 5.428484859985e-01 8.420345684

(array([0.9964257]), 0.996425704359249)

In [6]:
sc = connector.scattering_parameters(50*u.MHz)

In [7]:
ttc = sc.as_transfer_matrix()

In [8]:
sb = balun.scattering_parameters(50*u.MHz)

In [9]:
ttb = sb.as_transfer_matrix()

In [10]:
np.matmul(ttc[:, :, 0], ttb[:, :, 0])

array([[ 3.17118563e-01-0.946427j  ,  2.33632713e-03+0.01521592j],
       [-2.63824149e-04-0.01430951j,  3.18404696e-01+0.95014732j]])

In [11]:
np.matmul(ttc.transpose((2,0,1)), ttb.transpose((2,0,1))).transpose((1, 2, 0))

array([[[ 3.17118563e-01-0.946427j  ],
        [ 2.33632713e-03+0.01521592j]],

       [[-2.63824149e-04-0.01430951j],
        [ 3.18404696e-01+0.95014732j]]])

In [12]:
sb.s

array([[[0.01428786+0.00434318j],
        [0.36006532-0.93129072j]],

       [[0.36006532-0.93129072j],
        [0.01428786+0.00434318j]]])

In [13]:
both = sc.cascade_with(sb)

In [14]:
both.as_transfer_matrix()

array([[[ 3.17118563e-01-0.946427j  ],
        [ 2.33632713e-03+0.01521592j]],

       [[-2.63824149e-04-0.01430951j],
        [ 3.18404696e-01+0.95014732j]]])

In [15]:
sc.cascade_with(sb).as_transfer_matrix()

array([[[ 3.17118563e-01-0.946427j  ],
        [ 2.33632713e-03+0.01521592j]],

       [[-2.63824149e-04-0.01430951j],
        [ 3.18404696e-01+0.95014732j]]])

In [16]:
sb.cascade_with(sc).as_transfer_matrix()

array([[[ 3.17118563e-01-0.946427j  ],
        [ 2.63824149e-04+0.01430951j]],

       [[-2.33632713e-03-0.01521592j],
        [ 3.18404696e-01+0.95014732j]]])